In [6]:
# preprocessing
import gensim.downloader as api
import gensim.models

wiki_corpus = api.load('glove-wiki-gigaword-100')
twitter_corpus = api.load('glove-twitter-100')

In [7]:
def find_word(glove, a, b, x):
    # NOTE: slightly different from the Lab3, but same base code
    a = a.lower()
    b = b.lower()
    x = x.lower()
    out = f"> {a}:{b} as {x}:? \n"
    top_words = glove.most_similar_cosmul(positive=[x, b], negative=[a], topn=20)
    for num, (word, score) in enumerate(top_words):
        out += f"{num + 1}: ({score:.3f}) {word} \n"
    return out

In [8]:
def search_words(terms):
    for word in terms:
        try:
            if type(word) in [list, tuple]:
                print("Wiki:", find_word(wiki_corpus, *word))
                print("Twitter:", find_word(twitter_corpus, *word))
            else:
                print(f"---{word}---")
                print("Wiki: " + str(wiki_corpus.most_similar(word, topn=20)))
                print("Twitter: " + str(twitter_corpus.most_similar(word, topn=20)))
        except Exception as e:
            print(e)

# What kind of biases are present in either dataset, if any?

In [9]:
search_words(['man', 'woman', 'feminist', ('european', 'white', 'asian'), ('man', 'jock', 'woman'), ('woman', 'cheerleader', 'man')])

---man---
Wiki: [('woman', 0.8323495388031006), ('boy', 0.7914870977401733), ('one', 0.7788748741149902), ('person', 0.7526815533638), ('another', 0.752223551273346), ('old', 0.7409117221832275), ('life', 0.737169623374939), ('father', 0.7370322346687317), ('turned', 0.7347695231437683), ('who', 0.734551191329956), ('whose', 0.7326126098632812), ('girl', 0.7291691303253174), ('he', 0.7255576252937317), ('him', 0.7238516807556152), ('young', 0.7218634486198425), ('himself', 0.7214202284812927), ('friend', 0.7170529961585999), ('once', 0.7132790684700012), ('being', 0.7123121023178101), ('a', 0.7093364000320435)]
Twitter: [('boy', 0.7652449011802673), ('dude', 0.7523702383041382), ('guy', 0.7378775477409363), ('was', 0.7247804403305054), ("'s", 0.7206681966781616), ('bad', 0.7175806760787964), ('men', 0.7122883200645447), ('hell', 0.7033430337905884), ('shit', 0.7005720138549805), ('that', 0.695851743221283), ('is', 0.6952483654022217), ('he', 0.6951324343681335), ('but', 0.6918841004371

# Here we can see different biases present in the two corpora.
- Man:
   - Wikipedia: ageism "old"
   - Twitter: discrimination "bad"/"hell"/"shit"
- Woman:
   - Wikipedia: Gender Stereotype "mother" 3rd result
   - Twitter: Gender Stereotype "mother" 4th result 
- Feminist:
   - Wikipedia: Sexual Orientation Bias "lesbian"
- European : While = Asian : X
   - Wikipedia: "black" (might be just "overfitting" due to color white association, also other output are colors)
   - Twitter: Lookism "ugly"/"fat"
- Man : Jock = Woman : X
   - Twitter: Gender Bias "slutbucket" (perhaps the forced association "Man: Jock" messes up the output)
- Woman : Cheerleader = Man : X
   - Wikipedia: Stereotype "nerd"/"geek"

In the Woman <-> Mother association is possible to see a human perspective
The presented stereotypes show that Twitter, being community driven and less controlled then Wikipedia, presents more negative bias, for example the man discrimination or the gender bias for woman in relation with jock 

# Are there any differences with respect to bias between the two kinds of data?

In [10]:
search_words(['party', 'heroine', 'feet', 'assassin', 'us', 'rip'])

---party---
Wiki: [('opposition', 0.8269407153129578), ('democratic', 0.8042268753051758), ('parties', 0.7943356037139893), ('coalition', 0.7723221182823181), ('leader', 0.7596955299377441), ('election', 0.7480783462524414), ('candidate', 0.746288537979126), ('liberal', 0.7460210919380188), ('socialist', 0.7336400151252747), ('conservative', 0.7326690554618835), ('elections', 0.7202402949333191), ('ruling', 0.7180086374282837), ('nationalist', 0.7169246077537537), ('leadership', 0.7141717076301575), ('political', 0.7107859253883362), ('communist', 0.7097721099853516), ('democrats', 0.7013227939605713), ('vote', 0.7007361650466919), ('bjp', 0.6920799612998962), ('candidates', 0.6906204223632812)]
Twitter: [('saturday', 0.7777314186096191), ('friday', 0.7741458415985107), ('house', 0.7633090615272522), ('night', 0.7590202689170837), ('rave', 0.7588706016540527), ('parties', 0.7471879124641418), ('tonight', 0.741820216178894), ('next', 0.7226822376251221), ('coming', 0.7079103589057922), 

# Here we can see a difference between the twitter and the wikipedia corpora.
- Party:
   - Wikipedia: election party
   - Twitter: fun event
- Heroine:
   - Wikipedia: comic/movies hero
   - Twitter: drug
- Feet:
   - Wikipedia: measure
   - Twitter: body part
- Assassin:
   - Wikipedia: killer
   - Twitter: video games
- US:
   - Wikipedia: country
   - Twitter: pronoun
- RIP:
   - Wikipedia: tearing apart
   - Twitter: death
 
Wikipedia seems more data driven, is visible that the main purpose is to transfer knowledge.
 Twitter seems more user driven, is visible that the main purpose is to communicate with other users and present a personal point of view.

# Can biases between concepts be revealed trough visualization?

In [11]:
import gensim


def extract_word_vectors(word_list: list[str], corpus: gensim.models.KeyedVectors):
    word_vectors = []
    for word in word_list:
        try:
            word_vectors.append(corpus[word])
        except Exception as e:
            print(e)
    return word_list, word_vectors

In [15]:
import os


def save_word_vector_for_visualisation(word_list: list[str], word_vectors: list[list[float]], output_path: str):
    word_vector = gensim.models.KeyedVectors(vector_size=100)
    word_vector.add_vectors(word_list, word_vectors)
    if not os.path.exists(output_path) or not os.path.isdir(output_path):
        os.mkdir(output_path)
    word_vector.save_word2vec_format(output_path + '/word_vector.txt', binary=False)
    ! python -m gensim.scripts.word2vec2tensor --input {output_path}/word_vector.txt --output {output_path}/word_vector


In [17]:
sample_word_list, sample_word_vectors = extract_word_vectors(['pilot', 'teacher', 'surgeon', 'babysitter', 'engineer', 'hairdresser', 'man', 'woman'], wiki_corpus)
save_word_vector_for_visualisation(sample_word_list, sample_word_vectors, './output/wiki_output')
sample_word_list, sample_word_vectors = extract_word_vectors(['pilot', 'teacher', 'surgeon', 'babysitter', 'engineer', 'hairdresser', 'man', 'woman'], twitter_corpus)
save_word_vector_for_visualisation(sample_word_list, sample_word_vectors, './output/twitter_output')

2023-09-19 20:09:59,303 - word2vec2tensor - INFO - running /home/administrator/Desktop/T-725-MALV-Natural-Language-Processing/assignments/assignment_1/venv/lib/python3.10/site-packages/gensim/scripts/word2vec2tensor.py --input ./output/wiki_output/word_vector.txt --output ./output/wiki_output/word_vector
2023-09-19 20:09:59,303 - keyedvectors - INFO - loading projection weights from ./output/wiki_output/word_vector.txt
2023-09-19 20:09:59,306 - utils - INFO - KeyedVectors lifecycle event {'msg': 'loaded (8, 100) matrix of type float32 from ./output/wiki_output/word_vector.txt', 'binary': False, 'encoding': 'utf8', 'datetime': '2023-09-19T20:09:59.304736', 'gensim': '4.3.2', 'python': '3.10.13 (main, Aug 25 2023, 13:20:03) [GCC 9.4.0]', 'platform': 'Linux-5.4.0-163-generic-x86_64-with-glibc2.31', 'event': 'load_word2vec_format'}
2023-09-19 20:09:59,310 - word2vec2tensor - INFO - 2D tensor file saved to ./output/wiki_output/word_vector_tensor.tsv
2023-09-19 20:09:59,311 - word2vec2tensor

I think visualisation can help in finding some biases, but it's not a definitive solution.
I tried using [https://projector.tensorflow.org](https://projector.tensorflow.org) and I found some interesting results.
By inputting some gender-biased words, the plot shows clearly two areas, with a large margin and "male" and "female" usable as support vectors for defining a margin in an SVM like fashion.

# Can bias in word embeddings be removed? if so, how?

In my opinion is not possible or at least it is non-trivial to remove completely bias in word embeddings.
The main reason is that bias are part of the human thought process that is reflected in the text corpus.
And the word vectors are extracted analysing the text corpus.

Bias can also be seen as a way humans have to separate, logically connect and differentiate concepts.
For example, a basket ball and the planet are two different concepts, but is possible to logically relate both of them to a sphere.
This duality may be reflected in a word2vec approach, where, depending on the corpus used for training, a sphere may be more near to either a ball or a planet.

Also, is necessary to differentiate between positive and negative bias from a human perspective.
A positive bias can be something similar to the sphere <-> ball/planet relation, it is indeed unharmful and can be useful to explain and understand concepts.
A negative bias can be the "ugly"/"fat" or the "bad"/"hell" relation presented before.

In my opinion, the only way to completely remove bias is to remove the human factor from the text corpus, which is not possible due to the nature of the text corpus itself.
Is arguable that AI generated text corpora are not generated by humans, but the AI is trained on human generated text corpora, so the human factor may still be present.

I think, the bias can be lowered by using a more general or large text corpus, for example by using a text corpus that is not language specific, like the [Universal Dependencies](https://universaldependencies.org/).
Also, the bias can be lowered by analysing the generated model and mitigating the obtained results, just like the work performed by the OpenAI engineers, that created specific datasets to lower any bias and points of view from the GPT model.
Finally, a more mathematical way to approach the problem, is by creating a neutral vector and use it to identify the bias, then, by using the bias vector, is possible to remove the bias from the model.
Is important to notice that this behaviour may lead to a loss of information.
For example, words like "father" and "mother" if the gender is perceived as a bias, can be de-biased and the resulting vector may be something similar to "parent".
This way, the gender bias will be removed, but also the meaning of the word will change.
The change described may improve the results by lowering the presence of bias, but may eventually lead to other problems, like inconsistencies or incongruences in the model.
Therefore, the mathematical approach to a vector normalisation or de-biasing, has to be performed following some criterion in order to decide which words have to remain biased, and which words have to be neutral, thus leaving some "wanted" bias.